# core

> Fill in a module description here

In [1]:
#| default_exp core

In [2]:
#| hide
from nbdev.showdoc import *

## Variáveis numéricas

In [3]:
#| export
def func_query_janela_num(lista_janela, feat_num_lista, id, safra_ref, target, tb_feat, safra):
    query_janela = ""
    lista_vars = ""

    for n in lista_janela:
        query_variaveis_numericas=""
        for i in feat_num_lista:
            query_variaveis_numericas +=f"""
             -- Criação de variáveis numéricas a partir da coluna {i} para a janela {n}
            SUM(COALESCE({tb_feat}.{i},0)) AS {i}_SUM_{n}M,
            MIN(COALESCE({tb_feat}.{i},0)) AS {i}_MIN_{n}M,
            MAX(COALESCE({tb_feat}.{i},0)) AS {i}_MAX_{n}M,
            AVG(COALESCE({tb_feat}.{i},0)) AS {i}_AGV_{n}M,
            """
            lista_vars += f"""
            tb_join.{i}_SUM_{n}M,
            tb_join.{i}_MIN_{n}M,
            tb_join.{i}_MAX_{n}M,
            tb_join.{i}_AGV_{n}M,
            """
        query_variaveis_numericas = query_variaveis_numericas.rstrip(', \n')

        query_janela += f"""
        -- Criação de variáveis de janela de {n}M
        tb_janela_{n}M as(
            SELECT 
                tb_public.{id},
                tb_public.{safra_ref},
                tb_public.{target},
                {query_variaveis_numericas}

            FROM tb_public
            LEFT JOIN {tb_feat} 
            ON  tb_public.{id} = tb_feat.{id}
            AND (strftime('%Y-%m-%d', date({tb_feat}.{safra}, '+{n} months')) >= tb_public.{safra_ref})
                AND ({tb_feat}.{safra} < tb_public.{safra_ref})
            GROUP BY tb_public.{id}, tb_public.{safra_ref}
        ),
        """
    lista_vars = lista_vars.rstrip(', \n')
    return query_janela, lista_vars

In [4]:
#| export
def func_query_join_num(lista_janela, id, safra_ref):
    query_join= ""
    for i in lista_janela:
        query_join += f"""
        LEFT JOIN tb_janela_{i}M
            ON tb_public.{id} = tb_janela_{i}M.{id}
            AND tb_public.{safra_ref} = tb_janela_{i}M.{safra_ref}
    """
    return query_join

In [5]:
#| export
def create_query_num(tb_publico, tb_feat, lista_janela,feat_num_lista, id, safra_ref, target, safra):
    
    query_janela, lista_var = func_query_janela_num(lista_janela, feat_num_lista, id, safra_ref, target, tb_feat, safra)
    query_num = f"""
    WITH 
    tb_public AS (
        SELECT 
            *
        FROM {tb_publico}
    ),
    {query_janela}

    tb_join AS (
        SELECT 
            *
        FROM tb_public 
        {func_query_join_num(lista_janela, id, safra_ref)}
    )
        
    SELECT 
        tb_join.{id},
        tb_join.{safra_ref},
        tb_join.{target},
        {lista_var}
    FROM tb_join
    """
    return query_num

## Variáveis categóricas

In [6]:
#| export
def func_query_janela_cat(lista_janela, feat_cat_lista, id, safra_ref, target, tb_feat, safra):
  
    vars_cat = ""
    query_janela_cat = ""
    join_moda = ""
    for n in lista_janela:
        for i in feat_cat_lista:
            query_janela_cat += f"""
    tb_janela_{i}_{n}M as(
        SELECT
            tb_public.{id},
            tb_public.{safra_ref},
            tb_public.{target},
            {tb_feat}.{i},
            COUNT(*) AS frequency_{i}
        FROM tb_public
        LEFT JOIN {tb_feat}
        ON tb_public.{id} = {tb_feat}.{id}
            AND (strftime('%Y-%m-%d', date({tb_feat}.{safra}, '+{n} months')) >= tb_public.{safra_ref})
            AND ({tb_feat}.{safra} < tb_public.{safra_ref})
        GROUP BY tb_public.{id}, tb_public.{safra_ref}, {tb_feat}.{i}
    ),

    tb_row_{i}_{n}M as (
        SELECT 
            *,    
            ROW_NUMBER() OVER (
                PARTITION BY 
                    {id},
                    {safra_ref}        
                    ORDER BY frequency_{i} DESC
            ) as row_num_{i}_{n}M
        FROM tb_janela_{i}_{n}M
    ),
    
    tb_moda_{i}_{n}M AS(
        SELECT
            tb_row_{i}_{n}M .{id},
            tb_row_{i}_{n}M .{safra_ref},
            tb_row_{i}_{n}M.{target},
            tb_row_{i}_{n}M.{i} AS {i}_MODA_{n}M
        FROM tb_row_{i}_{n}M 
        WHERE row_num_{i}_{n}M = 1
    ),
"""
            vars_cat +=f"""
        tb_moda_{i}_{n}M.{i}_MODA_{n}M,
                 """
            join_moda +=f"""
    LEFT JOIN tb_moda_{i}_{n}M 
    ON tb_moda_{i}_{n}M.{id} = tb_public.{id}
    AND tb_moda_{i}_{n}M.{safra_ref} = tb_public.{safra_ref}
"""
    vars_cat = vars_cat.rstrip(', \n')
    query_janela_cat = query_janela_cat.rstrip(', \n')
    return query_janela_cat, vars_cat, join_moda

In [7]:
#| export
def create_query_cat(tb_publico, tb_feat, lista_janela,feat_cat_lista, id, safra_ref, target, safra):
    query_janela_cat, lista_vars, join_moda  = func_query_janela_cat(lista_janela, feat_cat_lista, id, safra_ref, target, tb_feat, safra)

    query_num_cat = f"""
    WITH 
    tb_public as (
        SELECT 
            {id},
            {safra_ref},
            {target}
        FROM {tb_publico}
    ),
    {query_janela_cat}

    SELECT 
        tb_public.{id},
        tb_public.{safra_ref},
        tb_public.{target},
        {lista_vars}
    FROM tb_public
    {join_moda}
    """
    return query_num_cat

In [8]:
#| hide
import nbdev; nbdev.nbdev_export()